In [3]:
import pandas as pd
import random
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

# Khởi tạo sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Dữ liệu mẫu với các comment giả định
data = {
    'product_id': [f'P{random.randint(100, 200)}' for _ in range(20)],
    'comment': [
        "Sản phẩm rất tốt, mình rất hài lòng.",
        "Không như mong đợi, sản phẩm quá tệ.",
        "Mình thích thiết kế của sản phẩm này!",
        "Chất lượng tạm ổn, giá hợp lý.",
        "Giao hàng nhanh, sản phẩm đúng như quảng cáo.",
        "Màu sắc không giống trong ảnh, thất vọng.",
        "Giá cả phải chăng, đáng mua.",
        "Đóng gói cẩn thận, nhưng chất lượng không tốt lắm.",
        "Hài lòng với sản phẩm này, sẽ mua lần nữa.",
        "Sản phẩm lỗi, không khuyến khích mua.",
        "Đẹp và chất lượng tốt!",
        "Quá đắt so với chất lượng nhận được.",
        "Mình sẽ giới thiệu sản phẩm này cho bạn bè.",
        "Hàng về bị móp, không hài lòng.",
        "Dịch vụ chăm sóc khách hàng kém.",
        "Nhận hàng nhanh chóng và chất lượng tuyệt vời.",
        "Màu sắc hợp thời trang.",
        "Không nên mua, chất lượng kém.",
        "Phù hợp với giá tiền, mình thấy ổn.",
        "Dịch vụ tuyệt vời và sản phẩm chất lượng."
    ]
}

# Tạo DataFrame
df = pd.DataFrame(data)

# Gán nhãn sentiment
def label_sentiment(comment):
    score = sia.polarity_scores(comment)
    if score['compound'] >= 0.05:
        return 'Tích cực'
    elif score['compound'] <= -0.05:
        return 'Tiêu cực'
    else:
        return 'Trung lập'

df['sentiment'] = df['comment'].apply(label_sentiment)

# Xuất dữ liệu ra tập dữ liệu đầu tiên - Dữ liệu gốc
df_real_data = df.copy()
df_real_data.to_csv('real_data.csv', index=False)

# Tạo tập dữ liệu 2: Gán nhãn thủ công theo xu hướng
def assign_preference(comment):
    if 'thích' in comment or 'hài lòng' in comment:
        return 'Sở thích'
    elif 'không' in comment or 'tệ' in comment or 'thất vọng' in comment:
        return 'Không thích'
    else:
        return 'Trung lập'

df['preference'] = df['comment'].apply(assign_preference)

# Xuất dữ liệu ra tập dữ liệu thứ hai - Dữ liệu gán nhãn
df_annotated_data = df.copy()
df_annotated_data.to_csv('annotated_data.csv', index=False)

print("Đã tạo thành công hai tập dữ liệu thử nghiệm.")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...


Đã tạo thành công hai tập dữ liệu thử nghiệm.


In [31]:
import pandas as pd
import re
import string
from vncorenlp import VnCoreNLP
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

# Khởi tạo sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Khởi tạo VnCoreNLP
annotator = VnCoreNLP("VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

# Đọc dữ liệu từ file
df = pd.read_csv('real_data.csv')

# 1. Loại bỏ các ký tự đặc biệt và chữ số
def remove_special_characters(text):
    text = re.sub(r'\d+', '', text)  # Loại bỏ số
    text = text.translate(str.maketrans('', '', string.punctuation))  # Loại bỏ dấu câu
    return text

df['cleaned_comment'] = df['comment'].apply(remove_special_characters)

# 2. Chuyển đổi thành chữ thường
df['cleaned_comment'] = df['cleaned_comment'].str.lower()

# 3. Định nghĩa bộ stopwords tiếng Việt
stop_words = set([
    "và", "là", "của", "có", "cho", "với", "từ", "như", "nhưng", "về", "mình", "này", "đó", "ở", "thì", "làm", 
    "nên", "ra", "khi", "để", "nếu", "sẽ", "các", "còn", "đến", "đang", "tại", "hay", "cùng", "hoặc"
    # Bổ sung thêm các từ stopwords tiếng Việt khác nếu cần
])

# 4. Loại bỏ stopwords và tách từ
def remove_stopwords(text):
    # Tách từ với VnCoreNLP
    words = annotator.tokenize(text)
    words = [w for sentence in words for w in sentence]  # Làm phẳng danh sách
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

df['cleaned_comment'] = df['cleaned_comment'].apply(remove_stopwords)

# 5. Gán nhãn sentiment cho comment đã được tiền xử lý
def label_sentiment(text):
    score = sia.polarity_scores(text)
    if score['compound'] >= 0.05:
        return 'Tích cực'
    elif score['compound'] <= -0.05:
        return 'Tiêu cực'
    else:
        return 'Trung lập'

df['sentiment'] = df['cleaned_comment'].apply(label_sentiment)

# 6. Lưu dữ liệu đã tiền xử lý
df.to_csv('preprocessed_data.csv', index=False)

print("Tiền xử lý dữ liệu hoàn tất và đã lưu vào preprocessed_data.csv")

# Đóng annotator sau khi sử dụng
annotator.close()


ModuleNotFoundError: No module named 'vncorenlp'